In [50]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re
import time

driver = webdriver.Chrome('chromedriver.exe')

In [51]:
# 각 선수들의 프로필 링크 수집 
urls = []
i = 1
driver.get("https://olympics.com/tokyo-2020/paralympic-games/en/results/all-sports/athletes.htm")

while i < 228:
    driver.implicitly_wait(5)
    
    elems = driver.find_elements_by_css_selector(".playerTag [href]")
    urls += [elem.get_attribute('href') for elem in elems]
    
    if i == 1:
        # 쿠키 허용
        driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
    
    try:
        # 팝업창 닫기
        driver.find_element_by_xpath('//*[@id="gdfr-initial-screen"]/div[2]/div[3]/a[1]').click()
    except NoSuchElementException:
        print("Keep Going~")
    
    try:
        path = '//*[@id="entries-table_next"]'
        driver.find_element_by_xpath(path).click()
    except:
        print('Error in {}'.format(i))
    
    i += 1

Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep Going~
Keep

In [61]:
len(set(urls))

4527

In [62]:
with open("athletes_profile.txt", "w") as file:
    file.write("\n".join(urls))

In [55]:
df = pd.DataFrame(columns=['name', 'country', 'sports', 'age', 'gender', 'philosophy', 'ambition', 'hero'])
count = 0

# 각 프로필 링크별 정보 수집
for url in urls:
    with urllib.request.urlopen(url) as url:
        try:
            name, country, sports, age, gender, philosophy, ambition, hero = None, None, None, None, None, None, None, None
            doc = url.read()
            soup = BeautifulSoup(doc, "html.parser")
            name = soup.find_all("h1")[0].get_text().replace('\r', '')
            country = soup.find_all("a", class_="country")[0].get_text()
            sports = soup.find_all("div", class_="row my-2")[0].get_text()
            for foo in soup.find_all("div", class_="col-md-6")[0]:
                try:
                    if "Age:" in str(foo).split("\r")[0]:
                        age = str(foo).split("\r")[1].split("</div>")[0]
                    if "Gender:" in str(foo).split("\r")[0]:
                        gender = str(foo).split("\r")[1].split("</div>")[0]            
                except:
                    pass
            for foo in soup.find_all("div", class_="form-group"):
                try:
                    label = foo.find('label').text
                    if label == "Philosophy":
                        philosophy = foo.find('div', attrs={'class': 'divBio'}).text.replace('\r', '')
                    elif label == "Ambition":
                        ambition = foo.find('div', attrs={'class': 'divBio'}).text.replace('\r', '')
                    elif label == "Hero":
                        hero = foo.find('div', attrs={'class': 'divBio'}).text.replace('\r', '')
                except:
                    pass

            df.loc[count] = [name, country, sports, age, gender, philosophy, ambition, hero]
        
        except:
            print(url)
            
        count += 1

df.to_csv('athletes.csv', index=False, encoding='utf-8')

In [58]:
df

,name,country,sports,age,gender,philosophy,ambition,hero
0,AAJIM Munkhbat,Mongolia,Judo,32,Male,None,None,None
1,ABARZA Alberto,Chile,Swimming,36,Male,"""Swimming is not an individual sport. Certainl...",To compete at the 2020 Paralympic Games in Tok...,"Chilean tennis player Marcelo Rios, Brazilian ..."
2,ABASLI Namig,Azerbaijan,Judo,23,Male,None,None,None
3,ABASSI Mostefa,Algeria,Wheelchair Basketball,43,Male,"""What doesn't kill you makes you stronger."" (A...",None,"US basketballer LeBron James. (Athlete, 26 Aug..."
4,ABBAD Abderraouf,Algeria,Wheelchair Basketball,34,Male,None,None,"His father. (Athlete, 26 Aug 2021)"
...,...,...,...,...,...,...,...,...
2,ZURABIANI Zurab,Georgia,Judo,21,Male,None,None,None
3,ZURBRUGG Lindsey,United States of America,Wheelchair Basketball,22,Female,"""My goal every day is to make another person h...",To compete at the 2020 Paralympic Games in Tok...,None
4,ZVINOWANDA Vimbai,Zimbabwe,Athletics,29,Female,None,None,"Zimbabwean swimmer Kirsty Coventry, Zimbabwean..."
5,ZWOUKHI Fathi,Tunisia,Triathlon,35,Male,None,None,None
